In [23]:
from abc import abstractmethod

class LLM:
    
    @abstractmethod
    def generate(self, system_prompt: str, user_prompt: str) -> str:
        """
        Generate an answer to prompt passed as a parameter
        
        Args:
            system_prompt: The prompt that defines the LLM
            user_prompt: The prompt to answer
        
        Returns:
            The output for that prompt
        """

class LLama3(LLM):

    def __init__(self):
        from dotenv import load_dotenv
        from groq import Groq
        import os

        load_dotenv()
        self.client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    
    def generate(self, system_prompt: str, user_prompt: str) -> str:
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ],
            model="llama3-70b-8192",
        )
        return chat_completion.choices[0].message.content

class Phi3(LLM):
    
    def __init__(self):
        import ollama
        self.client = ollama

    def generate(self, system_prompt: str, user_prompt: str) -> str:
        response = self.client.chat(model="phi3", messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ])
        return response["message"]["content"]

class FactoryLLM:
    
    @staticmethod
    def create_model(model_name: str) -> LLM:
        """
        Create a model based on the name passed as a parameter
        
        Args:
            model_name: The name of the model to create
        
        Returns:
            The model created
        """
        class_llm = {
            "llama3": LLama3,
            "phi3": Phi3
        }
        
        if model_name not in class_llm:
            raise ValueError(f"model must be in {class_llm.keys()}")
        
        return class_llm[model_name]()

model_name = "llama3"
model = FactoryLLM.create_model(model_name)

print(model.generate("Hola", "Hola"))

¡Hola! ¿Cómo estás?


In [24]:
# Zero-shot vs. few-shot

prompt_zero_shot = {
    "system_prompt": "Evalúa el sentimiento del texto que se te pase",
    "user_prompt": "No estoy seguro de cómo me siento."
}

prompt_zero_shot_improved = {
    "system_prompt": "Evalúa el sentimiento del texto que se te pase, "
              "Solo puedes responder con: Positivo, Negativo o Neutral"
              "La salida debe ser Sentimiento: <Positivo/Negativo/Neutral>",
    "user_prompt": "No estoy seguro de cómo me siento."
}

prompt_few_shot = {
    "system_prompt": "Texto: Estoy muy feliz hoy, Sentimiento: Positivo"
              "Texto: Odio tener que levantarme temprano, Sentimiento: Negativo"
              "Texto: No me importa si llueve Sentimiento: Neutral"
              "Evalúa el sentimiento del texto que se te pase usando los ejemplos anteriores",
    "user_prompt": "Texto: No estoy seguro de cómo me siento."
}

print(model.generate(**prompt_zero_shot))
print(model.generate(**prompt_zero_shot_improved))
print(model.generate(**prompt_few_shot))

El sentimiento predominante en este texto es la INCERTIDUMBRE o la DUDA. La frase expresa una sensación de no estar seguro o claro sobre cómo se siente, lo que sugiere un estado emocional de indecisión o confusión.
Sentimiento: Neutral
Sentimiento: Neutral


In [25]:
# Chain of thoughts

prompt_chain_of_thoughts = {
    "system_prompt": "Dado el problema que se te dé, debes resolverlo paso a paso y después dar una solución",
    "user_prompt": "¿Es 11436181919023821 un número primo?"
}

print(model.generate(**prompt_chain_of_thoughts))

Vamos a resolver este problema paso a paso:

**Paso 1: Definición de número primo**
Un número primo es un número natural mayor que 1 que solo tiene dos divisores: 1 y sí mismo.

**Paso 2: Análisis del número**
El número dado es 11436181919023821. Vamos a analizar sus propiedades.

**Paso 3: Verificar si es divisible por 2**
Si el número es par (acaba en 0, 2, 4, 6 o 8), no es primo. En este caso, el número termina en 1, por lo que no es divisible por 2.

**Paso 4: Verificar si es divisible por 3**
Si la suma de sus dígitos es divisible por 3, el número también lo es. La suma de los dígitos es:
1 + 1 + 4 + 3 + 6 + 1 + 8 + 1 + 9 + 0 + 2 + 3 + 8 + 2 + 1 = 40
La suma no es divisible por 3, por lo que el número no es divisible por 3.

**Paso 5: Verificar si es divisible por otros números primos**
Vamos a verificar si el número es divisible por otros números primos menores que su raíz cuadrada. La raíz cuadrada de 11436181919023821 es aproximadamente 338543.
Vamos a verificar la divisibilida

In [27]:
import time
import random

# Meta-Prompt Engineering

prompt_meta_prompt = {
    "system_prompt": "Tu misión es generar prompts que sean puedas resolver una tarea específica, se te dará el resultado de la anterior iteración para que puedas mejorar tu respuesta.",
    "user_prompt": "Determina si el número dado es primo. Anterior salida: {previous_output}"
}

previous_output = ""

for i in range(5):
    
    system_prompt = model.generate(**{
        "system_prompt": prompt_meta_prompt["system_prompt"],
        "user_prompt": prompt_meta_prompt["user_prompt"].format(previous_output=previous_output)
    })
    
    print(system_prompt)
    print(100*"-")
    
    number = random.randint(1, 1000000)
    prompt = {
        "system_prompt": system_prompt,
        "user_prompt": f"¿Es {number} un número primo?"
    }
    
    previous_output = model.generate(**prompt)
    time.sleep(3)

No hay anterior salida, así que empezaré con una respuesta inicial.

Aquí te dejo un prompt para determinar si un número dado es primo:

"Escribe un algoritmo que tome un número entero positivo como entrada y devuelva 'True' si el número es primo y 'False' en caso contrario."

Puedes proporcionarmefeedback sobre esta respuesta y mejoraré mi respuesta en la próxima iteración.
----------------------------------------------------------------------------------------------------
¡Excelente! Ahora que tienes un algoritmo para determinar si un número es primo, te propongo un desafío nuevo:

Determina si un número es un número de Fermat. Un número de Fermat es un número que se puede expresar como 2 elevado a una potencia de 2, más 1. Por ejemplo, 3 (2^2 + 1), 5 (2^4 + 1), 17 (2^8 + 1), etc.

Puedes utilizar el algoritmo de primalidad que acabamos de desarrollar para ayudarte en esta tarea. ¿Puedes escribir un algoritmo que determine si un número dado es un número de Fermat?
-------------------